## 1. Descripción General de las Cadenas Utilitarias

### *1.1. Resumir Documentos*

`load_summarize_chain`

In [2]:
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI

In [3]:
llm = ChatOpenAI(model= 'gpt-4o-mini', temperature= 0.9, max_tokens= 300)

In [5]:
# Leer el documento.
with open("document/muestra.txt", "r") as file:
    data = file.read()

Cuando se trata del procesamiento de documentos, es esencial dividir un documento grande en trozos más pequeños y manejables.

In [7]:
# Dividir texto.
divididor_texto = CharacterTextSplitter(
    separator= "\n",
    chunk_size= 1000,
    chunk_overlap= 200
)
dividido = divididor_texto.split_text(data)

In [9]:
# Crear multiples documentos.
documentos = [Document(page_content= page) for page in dividido]

In [10]:
documentos

[Document(metadata={}, page_content='India, oficialmente conocida como la República de la India, es un país diverso y vibrante ubicado en el sur de Asia. Con una rica historia milenaria, India es conocida por su patrimonio cultural, diversidad religiosa y vastos paisajes. Desde el majestuoso Himalaya en el norte hasta los serenos remansos de Kerala en el sur, India abarca una amplia gama de accidentes geográficos, incluyendo desiertos, llanuras, montañas y costas, lo que la convierte en una tierra de increíble belleza natural.\nLa India es el séptimo país más grande del mundo en cuanto a superficie y el segundo más poblado del mundo, con una población que supera los 1.300 millones de personas. Es una república democrática parlamentaria federal, con un presidente como jefe de Estado y un primer ministro como jefe de Gobierno. El país sigue una estructura administrativa de varios niveles, con 28 estados y 9 territorios de la unión, cada uno con su propio gobierno electo.'),
 Document(met

Para crear una instancia de `load_summarizer_chain`, debemos proporcionar tres argumentos.

- En primer lugar, debemos pasar el modelo de lenguaje grande deseado que se utilizará para consultar la entrada del usuario.

- En segundo lugar, especificamos el tipo de cadena langchain que se utilizará para resumir documentos.

- Por último, podemos establecer el argumento detallado en True si queremos ver todos los pasos intermedios involucrados en el procesamiento de la solicitud del usuario y la generación de la salida.

In [11]:
chain = load_summarize_chain(llm, chain_type= 'map_reduce', verbose= True)

In [13]:
chain.invoke(documentos)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"India, oficialmente conocida como la República de la India, es un país diverso y vibrante ubicado en el sur de Asia. Con una rica historia milenaria, India es conocida por su patrimonio cultural, diversidad religiosa y vastos paisajes. Desde el majestuoso Himalaya en el norte hasta los serenos remansos de Kerala en el sur, India abarca una amplia gama de accidentes geográficos, incluyendo desiertos, llanuras, montañas y costas, lo que la convierte en una tierra de increíble belleza natural.
La India es el séptimo país más grande del mundo en cuanto a superficie y el segundo más poblado del mundo, con una población que supera los 1.300 millones de personas. Es una república democrática parlamentaria federal, con un presidente como jefe de Estado y un primer ministro como jefe de Gobierno. El país sigue una estructura administrativa de

{'input_documents': [Document(metadata={}, page_content='India, oficialmente conocida como la República de la India, es un país diverso y vibrante ubicado en el sur de Asia. Con una rica historia milenaria, India es conocida por su patrimonio cultural, diversidad religiosa y vastos paisajes. Desde el majestuoso Himalaya en el norte hasta los serenos remansos de Kerala en el sur, India abarca una amplia gama de accidentes geográficos, incluyendo desiertos, llanuras, montañas y costas, lo que la convierte en una tierra de increíble belleza natural.\nLa India es el séptimo país más grande del mundo en cuanto a superficie y el segundo más poblado del mundo, con una población que supera los 1.300 millones de personas. Es una república democrática parlamentaria federal, con un presidente como jefe de Estado y un primer ministro como jefe de Gobierno. El país sigue una estructura administrativa de varios niveles, con 28 estados y 9 territorios de la unión, cada uno con su propio gobierno elec

### *1.2. Solicitudes HTTP*

`LLMRequestChain`

In [14]:
from langchain.chains import LLMRequestsChain, LLMChain

In [20]:
plantilla = """
Extrae la respuesta a la pregunta '{consulta}' o di "No encontrado" si la información no esta disponible.
{requests_result}
"""

indicacion_plantilla = PromptTemplate(
    input_variables= ['consulta', 'requests_result'],
    template= plantilla
)

In [21]:
chain = LLMRequestsChain(llm_chain= LLMChain(llm= llm, prompt= indicacion_plantilla))

Prepara la pregunta y entrada para la solicitud HTTP.

In [22]:
pregunta = '¿Cuál es la capital de Perú?'
entradas = {
    'consulta': pregunta,
    "url": "https://www.google.com/search?q=" + pregunta.replace(" ", "+")
} 

In [23]:
chain.invoke(entradas)

{'consulta': '¿Cuál es la capital de Perú?',
 'url': 'https://www.google.com/search?q=¿Cuál+es+la+capital+de+Perú?',
 'output': 'La capital de Perú es Lima.'}

In [24]:
# Veamos el interno funcionamiento de la cadena.
import inspect
print(inspect.getsource(chain._call))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        from bs4 import BeautifulSoup

        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        # Other keys are assumed to be needed for LLM prediction
        other_keys = {k: v for k, v in inputs.items() if k != self.input_key}
        url = inputs[self.input_key]
        res = self.requests_wrapper.get(url)
        # extract the text from the html
        soup = BeautifulSoup(res, "html.parser")
        other_keys[self.requests_key] = soup.get_text()[: self.text_length]
        result = self.llm_chain.predict(  # type: ignore[attr-defined]
            callbacks=_run_manager.get_child(), **other_keys
        )
        return {self.output_key: result}

